In [1]:
import os
import pandas as pd
import datetime as dt
from database_app import db, WalliStat, Campaign

C:\ProgramData\Anaconda3\envs\flask_env\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [2]:
db.create_all()

Create `hourly` Campaign starting the next full hour using the `id=0`

In [3]:
t = dt.datetime.now()    
next_hour = t.replace(second=0, microsecond=0, minute=0, hour=t.hour+1)
hourly = Campaign(id=0, title="hourly", start=next_hour, interval=dt.timedelta(seconds=3600))
hourly

Campaign(id:0, 'hourly' is active:None, start:2021-10-17 10:00:00, end:None, interval:1:00:00)

In [4]:
db.session.add(hourly)
db.session.commit()

In [5]:
fn = "raw_2021-07-25.csv"
date_str = fn.split(".")[0].split("_")[1]
df = pd.read_csv(os.path.join("..", "logs", fn))
df.tail()

,time,ver,charge_state,I_L1,I_L2,I_L3,Temp,V_L1,V_L2,V_L3,...,E_cyc_lb,E_hb,E_lb,I_max,I_min,watchdog,standby,remote_lock,max_I_cmd,FailSafe_I
6685,10:23:20,264,2,0,0,0,328,237,235,239,...,12088,2,61695,10,6,15000,4,1,100,10
6686,10:23:26,264,2,0,0,0,325,236,235,238,...,12088,2,61695,10,6,15000,4,1,100,10
6687,10:23:31,264,2,0,0,0,328,235,234,238,...,12088,2,61695,10,6,15000,4,1,100,10
6688,10:23:36,264,2,0,0,0,328,235,234,238,...,12088,2,61695,10,6,15000,4,1,100,10
6689,10:23:41,264,2,0,0,0,328,235,234,238,...,12088,2,61695,10,6,15000,4,1,100,10


In [6]:
df.describe()

,ver,charge_state,I_L1,I_L2,I_L3,Temp,V_L1,V_L2,V_L3,ext_lock,...,E_cyc_lb,E_hb,E_lb,I_max,I_min,watchdog,standby,remote_lock,max_I_cmd,FailSafe_I
count,6690.0,6690.000000,6690.000000,6690.000000,6690.000000,6690.000000,6690.000000,6690.000000,6690.000000,6690.0,...,6690.000000,6690.0,6690.000000,6690.0,6690.0,6690.0,6690.0,6690.0,6690.000000,6690.0
mean,264.0,5.008371,19.503737,17.402990,16.397459,292.886996,236.999253,235.891779,239.402242,1.0,...,10796.983558,2.0,60403.983558,10.0,6.0,15000.0,4.0,1.0,98.505232,10.0
std,0.0,1.402405,39.795441,36.834195,34.548114,18.596817,1.342131,1.429232,1.326984,0.0,...,3076.976561,0.0,3076.976561,0.0,0.0,0.0,0.0,0.0,12.135259,0.0
min,264.0,2.000000,0.000000,0.000000,0.000000,272.000000,233.000000,231.000000,236.000000,1.0,...,0.000000,2.0,49607.000000,10.0,6.0,15000.0,4.0,1.0,0.000000,10.0
25%,264.0,5.000000,0.000000,0.000000,0.000000,277.000000,236.000000,235.000000,238.000000,1.0,...,12088.000000,2.0,61695.000000,10.0,6.0,15000.0,4.0,1.0,100.000000,10.0
50%,264.0,5.000000,0.000000,0.000000,0.000000,283.000000,237.000000,236.000000,239.000000,1.0,...,12088.000000,2.0,61695.000000,10.0,6.0,15000.0,4.0,1.0,100.000000,10.0
75%,264.0,5.000000,0.000000,0.000000,0.000000,310.000000,238.000000,237.000000,240.000000,1.0,...,12088.000000,2.0,61695.000000,10.0,6.0,15000.0,4.0,1.0,100.000000,10.0
max,264.0,7.000000,106.000000,99.000000,92.000000,328.000000,240.000000,239.000000,243.000000,1.0,...,12088.000000,2.0,61695.000000,10.0,6.0,15000.0,4.0,1.0,100.000000,10.0


In [7]:
hourly.id

0

In [8]:
for index, row in df.head(6).iterrows():
    ws = WalliStat(datetime=pd.to_datetime(date_str + " " + row.time).to_pydatetime(), 
                   Temp=row.Temp/10., 
                   Power=row.P,
                   campaign_id=hourly.id)
    db.session.add(ws)
db.session.commit()

In [9]:
WalliStat.query.filter(WalliStat.Temp>1).all()

[WalliStat(id:1-->campaign.id:0, 2021-07-25 00:53:32: 27.2°C, 0W),
 WalliStat(id:2-->campaign.id:0, 2021-07-25 00:53:37: 27.2°C, 0W),
 WalliStat(id:3-->campaign.id:0, 2021-07-25 00:53:43: 27.2°C, 0W),
 WalliStat(id:4-->campaign.id:0, 2021-07-25 00:53:48: 27.2°C, 0W),
 WalliStat(id:5-->campaign.id:0, 2021-07-25 00:53:53: 27.2°C, 0W),
 WalliStat(id:6-->campaign.id:0, 2021-07-25 00:53:58: 27.2°C, 0W)]

In [10]:
campaign1 = Campaign(title="High frequency polling for error checking.", start=dt.datetime.now(), end=dt.datetime.now()+dt.timedelta(days=1), interval=dt.timedelta(seconds=1))
db.session.add(campaign1)
db.session.commit()

In [11]:
cmps = Campaign.query.all()
cmps

[Campaign(id:0, 'hourly' is active:True, start:2021-10-17 10:00:00, end:None, interval:1:00:00),
 Campaign(id:1, 'High frequency polling for error checking.' is active:True, start:2021-10-17 09:26:10.057643, end:2021-10-18 09:26:10.057643, interval:0:00:01)]

In [12]:
vars(campaign1)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1bfe24a0910>,
 'title': 'High frequency polling for error checking.',
 'start': datetime.datetime(2021, 10, 17, 9, 26, 10, 57643),
 'end': datetime.datetime(2021, 10, 18, 9, 26, 10, 57643),
 'measure_walli': True,
 'id': 1,
 'is_active': True,
 'interval': datetime.timedelta(seconds=1),
 'measure_light': True}

In [13]:
hourly.walli_stats

[WalliStat(id:1-->campaign.id:0, 2021-07-25 00:53:32: 27.2°C, 0W),
 WalliStat(id:2-->campaign.id:0, 2021-07-25 00:53:37: 27.2°C, 0W),
 WalliStat(id:3-->campaign.id:0, 2021-07-25 00:53:43: 27.2°C, 0W),
 WalliStat(id:4-->campaign.id:0, 2021-07-25 00:53:48: 27.2°C, 0W),
 WalliStat(id:5-->campaign.id:0, 2021-07-25 00:53:53: 27.2°C, 0W),
 WalliStat(id:6-->campaign.id:0, 2021-07-25 00:53:58: 27.2°C, 0W)]

In [16]:
WalliStat.query.first().campaign

Campaign(id:0, 'hourly' is active:True, start:2021-10-17 10:00:00, end:None, interval:1:00:00)

In [12]:
#db.drop_all()